# Quick Start (oda api v1.1.26)

In [1]:
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage,OdaLightCurve
from oda_api.data_products import BinaryData
import os

from importlib import reload

from oda_api.token import discover_token

import logging
logging.getLogger('oda_api').addHandler(logging.StreamHandler())
logging.getLogger().setLevel(logging.DEBUG)




## Connection to the dispatcher 

### build the dispatcher object

In [2]:
#TODO: change to prod
# disp=DispatcherAPI(url='https://frontend-staging.obsuks1.unige.ch/mmoda/dispatch-data',instrument='mock')
disp=DispatcherAPI(url='http://0.0.0.0:8001',instrument='mock')

 NO custom formatter for instrument mock 


### Get the list of instruments available on the gallery

In [3]:
import time
import jwt

secret_key = 'secretkey_test'
default_exp_time = int(time.time()) + 5000
token_payload = {
    'sub': "mtm@mtmco.net",
    'name': "mmeharga",
    'exp': default_exp_time,
    'tem': 0,
    'mstout': True,
    'mssub': True,
    'roles': 'general, gallery contributor'
}
token = jwt.encode(token_payload, secret_key, algorithm='HS256')

disp.get_list_terms_gallery(group='instruments', token=token)

Getting the list of available instruments on the gallery
List of terms from the group instruments successfully returned


['isgri',
 'jemx',
 'polar',
 'empty',
 'empty-async',
 'empty-semi-async',
 'spi_acs',
 'legacysurvey',
 'gw',
 'jemx1',
 'jemx2',
 'antares']

### Get the list of products available on the gallery

In [4]:
disp.get_list_terms_gallery(group='products', parent='development', token=token)

Getting the list of available instruments on the gallery
List of terms from the group products successfully returned


['numerical', 'dummy', 'parametrical']

### Get the list of sources available on the gallery

In [5]:
disp.get_list_terms_gallery(group='sources', token=token)

Getting the list of available instruments on the gallery
List of terms from the group sources successfully returned


['OAO 1657-415', 'GX 1+4', 'Cyg X-1', 'Crab']

In [6]:
instr_list=disp.get_instruments_list()
for i in instr_list:
    print (i)

dig_list unable to literal_eval empty; problem malformed node or string: <_ast.Name object at 0x7f1f4c2176a0>
dig_list unable to literal_eval empty-async; problem unexpected EOF while parsing (<unknown>, line 1)
dig_list unable to literal_eval empty-semi-async; problem unexpected EOF while parsing (<unknown>, line 1)


empty
empty-async
empty-semi-async


### get the description of the instrument

In [7]:
disp.get_instrument_description('empty')

unable to dig list, instance not a dict by <class 'list'>; object was [{'query_name': 'src_query'}, {'name': 'src_name', 'units': 'str', 'value': '1E 1740.7-2942'}, {'name': 'RA', 'units': 'deg', 'value': 265.97845833}, {'name': 'DEC', 'units': 'deg', 'value': -29.74516667}, {'name': 'T1', 'units': 'isot', 'value': '2017-03-06T13:26:48.000'}, {'name': 'T2', 'units': 'isot', 'value': '2017-03-06T15:32:27.000'}, {'name': 'token', 'units': 'str', 'value': None}]

--------------
query_name: src_query
 name: src_name,  value: 1E 1740.7-2942,  units: str, 
 name: RA,  value: 265.97845833,  units: deg, 
 name: DEC,  value: -29.74516667,  units: deg, 
 name: T1,  value: 2017-03-06T13:26:48.000,  units: isot, 
 name: T2,  value: 2017-03-06T15:32:27.000,  units: isot, 
 name: token,  value: None,  units: str, 
unable to dig list, instance not a dict by <class 'list'>; object was [{'query_name': 'empty_instrument_query'}, {'name': 'user_catalog', 'units': None, 'value': None}, {'name': 'p_list', 

[[{'instrumet': 'empty'},
  {'prod_dict': {'dummy': 'empty_parameters_dummy_query',
    'echo': 'echo_parameters_dummy_query',
    'failing': 'failing_parameters_dummy_query',
    'numerical': 'numerical_parameters_dummy_query',
    'parametrical': 'parametrical_parameters_dummy_query'}},
  '[{"query_name": "src_query"}, {"name": "src_name", "units": "str", "value": "1E 1740.7-2942"}, {"name": "RA", "units": "deg", "value": 265.97845833}, {"name": "DEC", "units": "deg", "value": -29.74516667}, {"name": "T1", "units": "isot", "value": "2017-03-06T13:26:48.000"}, {"name": "T2", "units": "isot", "value": "2017-03-06T15:32:27.000"}, {"name": "token", "units": "str", "value": None}]',
  '[{"query_name": "empty_instrument_query"}, {"name": "user_catalog", "units": None, "value": None}, {"name": "p_list", "units": None, "value": []}, {"name": "selected_catalog", "units": None, "value": None}]',
  '[{"query_name": "empty_parameters_dummy_query"}, {"product_name": "dummy"}]',
  '[{"query_name":

### We generate a dummy image

In [6]:
import time
import jwt

secret_key = 'secretkey_test'
default_exp_time = int(time.time()) + 5000
token_payload = {
    'sub': "mtm@mtmco.net",
    'name': "mmeharga",
    'exp': default_exp_time,
    'tem': 0,
    'mstout': True,
    'mssub': True,
    'roles': 'general, gallery contributor'
}
token = jwt.encode(token_payload, secret_key, algorithm='HS256')

data_collection=disp.get_product(instrument='empty',
                                 product='numerical',
                                 T1='2003-03-15T23:27:40.0',
                                 T2='2023-03-16T00:03:12.0',
                                 integral_data_rights="public",
                                 osa_version='OSA10.2',
                                 RA=50,
                                 DEC=45,
                                 max_pointings=1,
                                 detection_threshold=5.0,
                                 radius=15.,
                                 product_type='Dummy',
                                 token=token
                                )



please beware that by default, in a typical setup, oda_api will not output much. To learn how to increase the verbosity, please refer to the documentation: https://oda-api.readthedocs.io/en/latest/user_guide/ScienceWindowList.html?highlight=logging#Let's-get-some-logging . 
To disable this message you can pass `.get_product(..., silent=True)`
dig_list unable to literal_eval src_name; problem malformed node or string: <_ast.Name object at 0x7f1f9c056ac0>
dig_list unable to literal_eval RA; problem malformed node or string: <_ast.Name object at 0x7f1f4c244e20>
dig_list unable to literal_eval DEC; problem malformed node or string: <_ast.Name object at 0x7f1f4c244fd0>
dig_list unable to literal_eval T1; problem malformed node or string: <_ast.Name object at 0x7f1f4c244730>
dig_list unable to literal_eval T2; problem malformed node or string: <_ast.Name object at 0x7f1f4c1ac0a0>
dig_list unable to literal_eval token; problem malformed node or string: <_ast.Name object at 0x7f1f4c1ac220>
dig

### We upload the dummy product to the gallery applying some changes to some parameters

In [9]:
source_name='Mrk 421'

disp.resolve_source(source_name, token)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 54735.165023 from DATE-OBS.
Set MJD-END to 54735.638125 from DATE-END'. [astropy.wcs.wcs]
/home/gabriele/Workspace/oda_api/oda_api/plot_tools.py:129: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. This has been deprecated since 3.3 and in 3.6, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = mpl.cm.get_cmap("gist_earth").copy()
  self.cs.cmap.set_under('k')
Posting a product on the gallery
Getting the list of available instruments on the gallery
List of terms from the group instruments successfully returned

We noticed no instrument has been specified, the following instruments are available:
['isgri', 'jemx', 'polar', 'empty', 'empty-async', 'empty-semi-async', 'spi_acs', 'legacysurvey', 'gw', 'jemx1', 'jemx2', 'antares']
Please remember that this can be set at a later st

{'DEC': 38.208833, 'RA': 166.113808, 'name': 'Mrk 421', 'resolver': 'NED'}

In [12]:
image_product = oda_api.plot_tools.OdaImage(data_collection)
# img_fn = image_product.get_image_for_gallery()

# generate two dummy fits files
# fits_file_fn_1 = image_product.write_fits(file_prefix='first')
# fits_file_fn_2 = image_product.write_fits(file_prefix='second')

    
# token = discover_token()


insert_new_source=True
force_insert_new_source=True

d = disp.post_data_product_to_gallery(token=token,
                                      RA=55, DEC=46, e1_kev=20, e2_kev=40,
                                      instrument='empty', product_type='numerical',
                                      product_title="very nice title",
                                      src_name=source_name,
                                      validate_source=True,
                                      insert_new_source=insert_new_source,
                                      force_insert_new_source=force_insert_new_source,
#                                       gallery_image_path=img_fn,
#                                       fits_file_path=[fits_file_fn_1, fits_file_fn_2]
                                     )

Searching the object Mrk 421
 name: Mrk 421, 
source Mrk 421 validated using the service NED
Posting a product on the gallery
Product successfully posted on the gallery, at the link http://cdciweb02.isdc.unige.ch/mmoda/gallery/node/2308
